In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [24]:
import seaborn as sn
import warnings
warnings.filterwarnings("ignore")

In [25]:
#NLP
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

In [26]:
from PIL import Image

In [27]:
import re
import string

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.externals import joblib
from sklearn import linear_model
import sklearn.preprocessing
import sklearn.linear_model
import sklearn.pipeline

In [29]:
from PyQt5.QtCore import QAbstractTableModel, Qt
from PyQt5 import QtWidgets, uic, QtGui,QtCore
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QMessageBox,QDesktopWidget
from PyQt5.QtGui import QIcon
import sys
from PyQt5 import QtWidgets
import os
from PyQt5.QtWidgets import QMainWindow, QApplication
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from PyQt5.QtWidgets import QDialog, QApplication, QPushButton, QVBoxLayout,QMessageBox,QProgressBar

***Data set import:***

In [30]:
"""It is not necessary to re-rerun this cells after a %reset"""

"to load assets on the server (if it was not already done)"
import os
if not os.path.exists("Dataset"):
    !git clone https://github.com/youness-elh/Dataset

"checking: directories 'dataframe','img'  must appear"
!ls Dataset

Cloning into 'Dataset'...
fatal: unable to access 'https://github.com/youness-elh/Dataset/': Failed to connect to github.com port 443: Connection refused
Der Befehl "ls" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


# Objective:

***Food choices and preferences of college students***

Dans ce notebooke j'ai essayé de mettre en oeuvre l'ensemble des modèles `pickelés` dans le notebook `Food_choice_pipeline` sous forme d'interface graphique. l'idée est de faciliter l'étude des diffèrents sénarios possibles tout en rèpondant à plusieurs question. Pour l'instant l'application rèsume seulememt la rèponse à la question:

* Quelle est la relation entre les autres descipteurs et la note de l´êleve?

Sans interaction avec l'utilisateur...


# Pipeline de prétraitement et Interface graphique:

Notre input X est tous les descpteurs décrits auparavant et notre output Y est le ´GPA´. 

Nous avons enpaqueté tous les prétraitement dans un pipeline, qui sera aussi appliqué au jeu test (sauf que nous ne fiterons jamais sur test).

Les réesultats sont présentés sous forme de graphes pour le jeu ´fité´ et ensuite celui gardé pour le test.


In [31]:
class MyPipeline():
        
    """this method is automaticaly called we buid the object i.e. we write pipeline=MyPipeLine() """
    def __init__(self,path_to_data,fill_with= 'median'):
        
        self.path = path_to_data
        self.fill_with = fill_with
        
        self._scaler = sklearn.preprocessing.StandardScaler()
        self._was_fit = False
        self._was_fit_strat = False
        self._was_read = False
        self._was_cleaned = False
        
        self.X_names=None
        self.df = None
        self.strat_test_set = None
        self.strat_train_set = None
        self.test_set = None
        self.train_set = None
        
        '''output'''
        self.X_train = None
        self.X_test = None
        self.Y_train = None
        self.Y_test = None
           
    def read_data(self):
        self._was_read = True
        self.df = pd.read_csv(self.path)
        self.data = self.df.copy()
        
    def clean_data(self,df,feature,feature_type,fill_with = 'median'):
    
        def remove_point(x):
            if x =='.':
                return ''
            else:
                return x
    
        if df[feature].dtype == object :
            df[feature] = df[feature].str.replace('[a-z]|[A-Z]|\,|\"|\;|\'|\s','')
            df[feature] = df[feature].apply(remove_point)
            df[feature].replace(to_replace='',value=0,inplace=True)
            #print('In "'+str(feature)+'" feature some non sens words were deleted')

        if df[feature].isnull().any():
            df[feature].fillna(0,inplace=True)
            #print('In "'+str(feature)+'" feature some nan value were replaced by the '+ str(fill_with))

        else:
            #print('"'+str(feature)+'" feature was already cleaned!')
            pass

        df[feature] = df[feature].astype(feature_type)
        mean = round(df[feature].mean(),1) if fill_with =='mean' else round(df[feature].median(),1)
        df[feature].replace(to_replace=0,value=mean,inplace=True) 

        if feature == 'weight':
             df[feature] = df[feature].apply(lambda x : round(x/2.205,3))

        return df
    
    def All_no_nlp_clean(self):
        self._was_cleaned = True
        
        if self._was_read == False:
            self.read_data()
            
        try:
            open_ended_columns = ['comfort_food','comfort_food_reasons','diet_current','eating_changes','father_profession','mother_profession','fav_cuisine','food_childhood','healthy_meal','ideal_diet','meals_dinner_friend','type_sports']
            df_no_nlp = self.df.drop(open_ended_columns,axis=1)
        except Exception:
            pass
        features = df_no_nlp.columns
        n =len(features)
        for i,f in enumerate(features):
            #print('-----'+str(i+1)+'th feature begins out of '+str(n)+' features-----')
            df_no_nlp = self.clean_data(df_no_nlp,f,'float','median')

        self.df = df_no_nlp
            
    def delete_corr_feat(self):
        try:
            self.df.drop("comfort_food_reasons_coded", axis=1, inplace=True)
        except Exception:
            pass
        
    def cleaned_nlp(self,feature = 'comfort_food'):
        open_ended_columns = ['comfort_food','comfort_food_reasons','diet_current','eating_changes','father_profession','mother_profession','fav_cuisine','food_childhood','healthy_meal','ideal_diet','meals_dinner_friend','type_sports']
        df_nlp = self.data.loc[:,open_ended_columns]
        
        def clean_text(text):
            '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
            text = text.lower()
            text = re.sub('\[.*?\]', '', text)
            text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
            text = re.sub('\w*\d\w*', '', text)
            text = re.sub('[‘’“”…]', '', text)
            text = re.sub('\n', '', text)
            return text
    
        for f in open_ended_columns:
            df_nlp[f] = pd.DataFrame(df_nlp[f].apply(lambda x: clean_text(str(x))))
            
        cv = CountVectorizer(stop_words='english')
        data_cv = cv.fit_transform(df_nlp[feature])
        data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
        data_dtm.index = df_nlp.index
        
        return df_nlp, data_dtm  
    
    def plot_top_words(self,feature = 'comfort_food'):
        df_nlp, data_dtm = self.cleaned_nlp(feature)
        
        try:
            maskComfy = 255-np.array(Image.open( "../heart.png"))
        except Exception:
            print('mask not available')
            pass

        wordcloud = (WordCloud(width=1440, height=1080, relative_scaling=0.5,
        stopwords=stopwords,mask=maskComfy,max_words=1000,background_color='white').generate_from_frequencies(df_nlp[feature].value_counts()))

        fig = plt.figure(1,figsize=(15, 15))
        plt.imshow(wordcloud,interpolation="gaussian")
        plt.axis('off')
        plt.show()
        
    
    def convert_to_cat(self):
        
        features = ['calories_chicken','calories_scone', 'tortilla_calories', 'turkey_calories','waffle_calories']
        def change_to_cat(x):
            if x == 265.0 or x == 107.0 or x == 580.0 or x == 345.0  or x == 575.0:
                x =1
            elif x == 430.0 or x == 315.0  or x == 725.0 or x == 500.0  or x == 760.0:
                x=2
            elif x == 610.0 or x == 420.0  or x == 940.0 or x == 690.0  or x == 900.0:
                x=3
            elif x == 720.0 or x == 980.0  or x == 1165.0 or x == 850.0  or x == 1315.0:
                x=4
            return int(x)

        for _,f in enumerate(features):
            self.df[f] = self.df[f].apply(change_to_cat)
            self.df[f] = self.df[f].astype('int')
        
    
    def standardize(self):
        num_columns = ['GPA','weight']
        scaler=StandardScaler()
        df_num_st=scaler.fit_transform(self.df[num_columns])
        self.df[num_columns] = pd.DataFrame(df_num_st)
        
    def x_train_test_random(self):
        if not self._was_fit:
            self.train_set, self.test_set = train_test_split(self.df, test_size=0.2, random_state=42)
        
    '''Dans notre jeu de donnée, le 'GPA' (Note) est un descripteur naturellement pour stratifier la population.''' 
    def x_train_test_strat(self, plot=False):
        
        if not self._was_fit_strat:
        
            self.df["GPA_cat"] = np.ceil(self.df["GPA"]/1.5)
            self.df["GPA_cat"].where(self.df["GPA"] < 4,4.0, inplace=True)
            split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
            for train_index, test_index in split.split(self.df, self.df["GPA_cat"]):
                self.strat_train_set = self.df.loc[train_index]
                self.strat_test_set = self.df.loc[test_index]
                
            if plot :
                freq=self.df["GPA_cat"].value_counts()
                # ou bien avec numpy
                # freq=np.unique(housing_0["income_cat"].values,return_counts=True)
                plt.bar(freq.index,freq.values);
                plt.show();

            self._was_fit_strat = True
            try:
                for set_ in (self.strat_train_set, self.strat_test_set):
                    set_.drop("GPA_cat", axis=1, inplace=True)
                    
                self.df.drop("GPA_cat", axis=1, inplace=True)
            except Exception:
                pass

        
    def input_output_numpy(self,feature_y = 'GPA',strat=True,conv_stand = True):
        
        if not self._was_read:
            self.read_data()
            #print('read')

        if not self._was_cleaned:
            self.All_no_nlp_clean()
            #print('cleaned')
         
        if conv_stand:
            '''convert to cat and standardize'''
            self.convert_to_cat()
            self.standardize()
            
        if strat == True:
            self.x_train_test_strat(False)
            self.X_test = self.strat_test_set.copy()
            self.X_train = self.strat_train_set.copy()
            
        else:
            self.x_train_test_random()
            self.X_test = self.test_set
            self.X_train = self.train_set
            
            
        self.Y_train = self.X_train[feature_y].values.reshape(-1)
        self.Y_test = self.X_test[feature_y].values.reshape(-1)
        
        self.X_train = self.X_train.drop(feature_y, axis=1, inplace=False)
        self.X_names = self.X_train.columns.values
        self.X_train = np.array(self.X_train)
        self.X_test = self.X_test.drop(feature_y, axis=1, inplace=False)
        self.X_test = np.array(self.X_test) 
        #print(self.Y_test.shape,self.X_test.shape)

## tests

In [32]:
'''
path_to_data = '..//food_coded.csv'
fill_with = 'median'
obj = MyPipeline(path_to_data ,fill_with)

obj.input_output_numpy()

X_train,Y_train=obj.X_train,obj.Y_train
X_test,Y_test=obj.X_test,obj.Y_test
X_names=obj.X_names

""" we check that MyPipeline.input_output_numpy() make the same job as our exploratory job."""
print('--------X_test-------- \n',X_test[:5,:10])
print('--------X_train-------- \n',X_train[:5,:10])
print("X_names\n",X_names[:10])
'''

'\npath_to_data = \'..//food_coded.csv\'\nfill_with = \'median\'\nobj = MyPipeline(path_to_data ,fill_with)\n\nobj.input_output_numpy()\n\nX_train,Y_train=obj.X_train,obj.Y_train\nX_test,Y_test=obj.X_test,obj.Y_test\nX_names=obj.X_names\n\n""" we check that MyPipeline.input_output_numpy() make the same job as our exploratory job."""\nprint(\'--------X_test-------- \n\',X_test[:5,:10])\nprint(\'--------X_train-------- \n\',X_train[:5,:10])\nprint("X_names\n",X_names[:10])\n'

In [33]:
class Plot_Window(QDialog):
    def __init__(self, parent=None):
        super(Plot_Window, self).__init__()
        self.data = parent
        self.constructor() 
        # Just some button connected to `plot` method
        self.button = QPushButton('Plot')
        self.button.clicked.connect(self.gui_plot)

        # set the layout
        layout = QVBoxLayout()
        layout.addWidget(self.toolbar)
        layout.addWidget(self.canvas)
        layout.addWidget(self.button)
        #layout.addWidget(self.button1)
        self.setLayout(layout)
        self.setWindowTitle('Plot')
        
        #self.show()
        
    def constructor(self):
        
        ''' To which purpose?  '''
        box = QMessageBox()
        box.setIcon(QMessageBox.Question)
        box.setWindowTitle('Figure size!')
        box.setStandardButtons(QMessageBox.Yes)
        Default = box.button(QMessageBox.Yes)
        Default.setText('Default')
        box.exec_()

        if box.clickedButton() == Default:
        # YES pressed
            # a figure instance to plot on
            self.figure = plt.figure()
            self.afont = {'fontname':'Arial'}
            self.tfont = self.afont#{'fontname':'Arial'}
            self.canvas = FigureCanvas(self.figure)

            self.toolbar = NavigationToolbar(self.canvas, self)
        
    def gui_plot(self):

        # instead of ax.hold(False)
        self.figure.clear()

        # create an axis
        ax = self.figure.add_subplot(111)

        # plot data
        self.data.plot(ax=ax);

        # refresh canvas
        self.figure.tight_layout()
        self.canvas.draw();
        plt.close()

In [34]:
class main_window_gui(QMainWindow):
    def __init__(self,path,fill_with):
        
        self.path = path
        self.fill_with = fill_with
        
        '''Load the ui'''
        QMainWindow.__init__(self)
        self.call = uic.loadUi('mainwindow_0.ui', self) 
        
        '''Linear regression'''
        self.lin1 = self.call.action_algo
        self.lin1.triggered.connect(self.Algo1)
        
        '''decision tree'''
        self.tree = self.call.action_tree
        self.tree.triggered.connect(self.Algo2)
        
        '''random forest'''
        self.forest = self.call.action_forest
        self.forest.triggered.connect(self.Algo3)

        self.show()
        
    def Algo1(self):
        obj = MyPipeline(self.path ,self.fill_with)
        obj.input_output_numpy()
        
        X_train,Y_train=obj.X_train,obj.Y_train
        X_test,Y_test=obj.X_test,obj.Y_test
        X_names=obj.X_names
        
        #try:
        best_model = joblib.load("my_model_linear.pkl") 
        Y_test_hat = best_model.predict(X_test)
        df=pd.DataFrame({"Y_test_hat_lin":Y_test_hat[:],"Y_test":Y_test[:]})
        window = Plot_Window(df)
        window.show()
        Y_train_hat = best_model.predict(X_train)
        df=pd.DataFrame({"Y_train_hat_lin":Y_train_hat[:],"Y_train":Y_train[:]})
        window = Plot_Window(df)
        window.show()
        #except Exception:
        #buttonReply = QMessageBox.critical(self, 
        #'Error message', "File could not be open")
                             
    def Algo2(self):
        obj = MyPipeline(self.path ,self.fill_with)
        obj.input_output_numpy()
        
        X_train,Y_train=obj.X_train,obj.Y_train
        X_test,Y_test=obj.X_test,obj.Y_test
        X_names=obj.X_names
        
        try:
            best_model = joblib.load("my_model_tree.pkl") 
            Y_test_hat = best_model.predict(X_test)
            df=pd.DataFrame({"Y_test_hat_tree":Y_test_hat[:],"Y_test":Y_test[:]})
            window = Plot_Window(df)
            window.show()
            Y_train_hat = best_model.predict(X_train)
            df=pd.DataFrame({"Y_train_hat_tree":Y_train_hat[:],"Y_train":Y_train[:]})
            window = Plot_Window(df)
            window.show()
        except Exception:
            buttonReply = QMessageBox.critical(self, 
            'Error message', "File could not be open")
    def Algo3(self):
        obj = MyPipeline(self.path ,self.fill_with)
        obj.input_output_numpy()
        
        X_train,Y_train=obj.X_train,obj.Y_train
        X_test,Y_test=obj.X_test,obj.Y_test
        X_names=obj.X_names
        
        try:
            best_model = joblib.load("my_model_forest.pkl") 
            Y_test_hat = best_model.predict(X_test)
            df=pd.DataFrame({"Y_test_hat_forest":Y_test_hat[:],"Y_test":Y_test[:]})
            window = Plot_Window(df)
            window.show()
            Y_train_hat = best_model.predict(X_train)
            df=pd.DataFrame({"Y_train_hat_forest":Y_train_hat[:],"Y_train":Y_train[:]})
            window = Plot_Window(df)
            window.show()
        except Exception:
            buttonReply = QMessageBox.critical(self, 
            'Error message', "File could not be open")
            
                

In [35]:
def main(path,fill_with):
    app = QtWidgets.QApplication(sys.argv) 
    window = main_window_gui(path,fill_with)
    app.exec_() 

In [36]:
path = '..//food_coded.csv'
fill_with = 'median'
main(path,fill_with);